In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset\Hackerearth Airplane Accident/train.csv")
df_test = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset\Hackerearth Airplane Accident/test.csv")

In [3]:
df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [4]:
y = df["Severity"]
df = df.drop("Severity",axis =1)

In [5]:
df = df.drop("Accident_ID",axis = 1)
ID = df_test["Accident_ID"]
df_test = df_test.drop("Accident_ID",axis = 1)

In [6]:
df.shape

(10000, 10)

In [7]:
df_test.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696


In [8]:
df_test.shape

(2500, 10)

In [9]:
combined = pd.concat([df,df_test],axis=0)

In [10]:

for i in combined.columns:
    temp = pd.DataFrame(combined[i])
    print(temp.nunique())

Safety_Score    1195
dtype: int64
Days_Since_Inspection    23
dtype: int64
Total_Safety_Complaints    44
dtype: int64
Control_Metric    963
dtype: int64
Turbulence_In_gforces    1128
dtype: int64
Cabin_Temperature    964
dtype: int64
Accident_Type_Code    7
dtype: int64
Max_Elevation    4447
dtype: int64
Violations    6
dtype: int64
Adverse_Weather_Metric    4556
dtype: int64


In [11]:
combined["Violations"] = combined["Violations"].astype("category")
combined["Accident_Type_Code"] = combined["Accident_Type_Code"].astype("category")


In [12]:
combined_dummy = pd.get_dummies(combined)

In [13]:
dum_df = combined_dummy.iloc[:10000,:]
dum_df_test = combined_dummy.iloc[10000:,:]

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(dum_df)
normal = scaler.transform(dum_df)
normal_test = scaler.transform(dum_df_test)

In [15]:
normal.shape

(10000, 21)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normal, y, test_size=0.001, random_state=42)

In [17]:
y_train = pd.get_dummies(y_train)

In [50]:
y_train.head()

,Highly_Fatal_And_Damaging,Minor_Damage_And_Injuries,Significant_Damage_And_Fatalities,Significant_Damage_And_Serious_Injuries
2750,0,0,0,1
7487,0,0,0,1
5272,1,0,0,0
5653,0,1,0,0
3999,1,0,0,0


In [18]:
import tensorflow as tf
from tensorflow import keras

In [29]:
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,BatchNormalization,Dropout


In [89]:
model = Sequential()
model.add(Dense(21,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = l2(0.01)))
model.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = l2(0.01)))
model.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = l2(0.01)))
model.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = l2(0.01)))
model.add(Dense(4,activation=tf.nn.softmax))

In [95]:
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [96]:
model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['categorical_crossentropy'])

In [97]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

In [98]:
# try learning rate on plateau as it needs to be decreased after 500 epochs or so (batch 128)

In [99]:
model.fit(X_train.values, y_train.values, epochs=500,batch_size = 128,validation_split = 0.05)

Train on 9490 samples, validate on 500 samples
Epoch 1/500
9490/9490 [==============================] - 1s 83us/sample - loss: 0.3158 - categorical_crossentropy: 0.1860 - val_loss: 0.3194 - val_categorical_crossentropy: 0.1898
Epoch 2/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3139 - categorical_crossentropy: 0.1843 - val_loss: 0.3186 - val_categorical_crossentropy: 0.1890
Epoch 3/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3139 - categorical_crossentropy: 0.1844 - val_loss: 0.3210 - val_categorical_crossentropy: 0.1914
Epoch 4/500
9490/9490 [==============================] - 0s 18us/sample - loss: 0.3131 - categorical_crossentropy: 0.1835 - val_loss: 0.3212 - val_categorical_crossentropy: 0.1918
Epoch 5/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3137 - categorical_crossentropy: 0.1842 - val_loss: 0.3221 - val_categorical_crossentropy: 0.1926
Epoch 6/500
9490/9490 [==============================

9490/9490 [==============================] - 0s 19us/sample - loss: 0.3132 - categorical_crossentropy: 0.1836 - val_loss: 0.3198 - val_categorical_crossentropy: 0.1901
Epoch 47/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3127 - categorical_crossentropy: 0.1830 - val_loss: 0.3173 - val_categorical_crossentropy: 0.1877
Epoch 48/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3122 - categorical_crossentropy: 0.1826 - val_loss: 0.3180 - val_categorical_crossentropy: 0.1884
Epoch 49/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3134 - categorical_crossentropy: 0.1837 - val_loss: 0.3170 - val_categorical_crossentropy: 0.1874
Epoch 50/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3133 - categorical_crossentropy: 0.1836 - val_loss: 0.3165 - val_categorical_crossentropy: 0.1870
Epoch 51/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3130 - categorical_crosse

9490/9490 [==============================] - 0s 36us/sample - loss: 0.3122 - categorical_crossentropy: 0.1828 - val_loss: 0.3184 - val_categorical_crossentropy: 0.1891
Epoch 92/500
9490/9490 [==============================] - 0s 34us/sample - loss: 0.3125 - categorical_crossentropy: 0.1831 - val_loss: 0.3221 - val_categorical_crossentropy: 0.1926
Epoch 93/500
9490/9490 [==============================] - 0s 37us/sample - loss: 0.3126 - categorical_crossentropy: 0.1832 - val_loss: 0.3172 - val_categorical_crossentropy: 0.1879
Epoch 94/500
9490/9490 [==============================] - 0s 37us/sample - loss: 0.3122 - categorical_crossentropy: 0.1827 - val_loss: 0.3197 - val_categorical_crossentropy: 0.1902
Epoch 95/500
9490/9490 [==============================] - 0s 35us/sample - loss: 0.3118 - categorical_crossentropy: 0.1824 - val_loss: 0.3160 - val_categorical_crossentropy: 0.1867
Epoch 96/500
9490/9490 [==============================] - 0s 35us/sample - loss: 0.3122 - categorical_crosse

9490/9490 [==============================] - 0s 20us/sample - loss: 0.3115 - categorical_crossentropy: 0.1823 - val_loss: 0.3152 - val_categorical_crossentropy: 0.1860
Epoch 137/500
9490/9490 [==============================] - 0s 21us/sample - loss: 0.3115 - categorical_crossentropy: 0.1823 - val_loss: 0.3193 - val_categorical_crossentropy: 0.1901
Epoch 138/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3114 - categorical_crossentropy: 0.1822 - val_loss: 0.3245 - val_categorical_crossentropy: 0.1953
Epoch 139/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3123 - categorical_crossentropy: 0.1832 - val_loss: 0.3168 - val_categorical_crossentropy: 0.1877
Epoch 140/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3122 - categorical_crossentropy: 0.1830 - val_loss: 0.3196 - val_categorical_crossentropy: 0.1904
Epoch 141/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3119 - categorical_c

Epoch 181/500
9490/9490 [==============================] - 0s 37us/sample - loss: 0.3117 - categorical_crossentropy: 0.1829 - val_loss: 0.3175 - val_categorical_crossentropy: 0.1887
Epoch 182/500
9490/9490 [==============================] - 0s 37us/sample - loss: 0.3117 - categorical_crossentropy: 0.1829 - val_loss: 0.3156 - val_categorical_crossentropy: 0.1867
Epoch 183/500
9490/9490 [==============================] - 0s 30us/sample - loss: 0.3112 - categorical_crossentropy: 0.1823 - val_loss: 0.3154 - val_categorical_crossentropy: 0.1866
Epoch 184/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3117 - categorical_crossentropy: 0.1828 - val_loss: 0.3167 - val_categorical_crossentropy: 0.1879
Epoch 185/500
9490/9490 [==============================] - 0s 19us/sample - loss: 0.3114 - categorical_crossentropy: 0.1826 - val_loss: 0.3189 - val_categorical_crossentropy: 0.1900
Epoch 186/500
9490/9490 [==============================] - 0s 18us/sample - loss: 0.3112 -

...

9490/9490 [==============================] - 0s 38us/sample - loss: 0.3100 - categorical_crossentropy: 0.1818 - val_loss: 0.3178 - val_categorical_crossentropy: 0.1898
Epoch 317/500
9490/9490 [==============================] - 0s 39us/sample - loss: 0.3101 - categorical_crossentropy: 0.1820 - val_loss: 0.3166 - val_categorical_crossentropy: 0.1883
Epoch 318/500
9490/9490 [==============================] - 0s 39us/sample - loss: 0.3107 - categorical_crossentropy: 0.1825 - val_loss: 0.3143 - val_categorical_crossentropy: 0.1862
Epoch 319/500
9490/9490 [==============================] - 0s 38us/sample - loss: 0.3112 - categorical_crossentropy: 0.1831 - val_loss: 0.3164 - val_categorical_crossentropy: 0.1883
Epoch 320/500
9490/9490 [==============================] - 0s 39us/sample - loss: 0.3105 - categorical_crossentropy: 0.1822 - val_loss: 0.3214 - val_categorical_crossentropy: 0.1931
Epoch 321/500
9490/9490 [==============================] - 0s 38us/sample - loss: 0.3105 - categorical_c

9490/9490 [==============================] - 0s 30us/sample - loss: 0.3110 - categorical_crossentropy: 0.1831 - val_loss: 0.3163 - val_categorical_crossentropy: 0.1884
Epoch 362/500
9490/9490 [==============================] - 0s 30us/sample - loss: 0.3111 - categorical_crossentropy: 0.1832 - val_loss: 0.3143 - val_categorical_crossentropy: 0.1864
Epoch 363/500
9490/9490 [==============================] - 0s 32us/sample - loss: 0.3105 - categorical_crossentropy: 0.1826 - val_loss: 0.3166 - val_categorical_crossentropy: 0.1887
Epoch 364/500
9490/9490 [==============================] - 0s 30us/sample - loss: 0.3101 - categorical_crossentropy: 0.1822 - val_loss: 0.3170 - val_categorical_crossentropy: 0.1891
Epoch 365/500
9490/9490 [==============================] - 0s 29us/sample - loss: 0.3097 - categorical_crossentropy: 0.1819 - val_loss: 0.3177 - val_categorical_crossentropy: 0.1899
Epoch 366/500
9490/9490 [==============================] - 0s 29us/sample - loss: 0.3102 - categorical_c

9490/9490 [==============================] - 0s 29us/sample - loss: 0.3102 - categorical_crossentropy: 0.1826 - val_loss: 0.3143 - val_categorical_crossentropy: 0.1867
Epoch 407/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3102 - categorical_crossentropy: 0.1826 - val_loss: 0.3200 - val_categorical_crossentropy: 0.1925
Epoch 408/500
9490/9490 [==============================] - 0s 29us/sample - loss: 0.3101 - categorical_crossentropy: 0.1826 - val_loss: 0.3152 - val_categorical_crossentropy: 0.1877
Epoch 409/500
9490/9490 [==============================] - 0s 29us/sample - loss: 0.3095 - categorical_crossentropy: 0.1820 - val_loss: 0.3144 - val_categorical_crossentropy: 0.1868
Epoch 410/500
9490/9490 [==============================] - 0s 29us/sample - loss: 0.3100 - categorical_crossentropy: 0.1825 - val_loss: 0.3172 - val_categorical_crossentropy: 0.1897
Epoch 411/500
9490/9490 [==============================] - 0s 28us/sample - loss: 0.3100 - categorical_c

9490/9490 [==============================] - 0s 28us/sample - loss: 0.3089 - categorical_crossentropy: 0.1815 - val_loss: 0.3132 - val_categorical_crossentropy: 0.1858
Epoch 452/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3100 - categorical_crossentropy: 0.1825 - val_loss: 0.3191 - val_categorical_crossentropy: 0.1917
Epoch 453/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3097 - categorical_crossentropy: 0.1823 - val_loss: 0.3152 - val_categorical_crossentropy: 0.1878
Epoch 454/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3098 - categorical_crossentropy: 0.1824 - val_loss: 0.3157 - val_categorical_crossentropy: 0.1883
Epoch 455/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3096 - categorical_crossentropy: 0.1822 - val_loss: 0.3177 - val_categorical_crossentropy: 0.1903
Epoch 456/500
9490/9490 [==============================] - 0s 26us/sample - loss: 0.3096 - categorical_c

9490/9490 [==============================] - 0s 21us/sample - loss: 0.3100 - categorical_crossentropy: 0.1827 - val_loss: 0.3151 - val_categorical_crossentropy: 0.1879
Epoch 497/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3092 - categorical_crossentropy: 0.1820 - val_loss: 0.3165 - val_categorical_crossentropy: 0.1893
Epoch 498/500
9490/9490 [==============================] - 0s 22us/sample - loss: 0.3100 - categorical_crossentropy: 0.1828 - val_loss: 0.3138 - val_categorical_crossentropy: 0.1865
Epoch 499/500
9490/9490 [==============================] - 0s 21us/sample - loss: 0.3092 - categorical_crossentropy: 0.1820 - val_loss: 0.3133 - val_categorical_crossentropy: 0.1861
Epoch 500/500
9490/9490 [==============================] - 0s 20us/sample - loss: 0.3094 - categorical_crossentropy: 0.1822 - val_loss: 0.3212 - val_categorical_crossentropy: 0.1941


In [100]:
nn_pred = model.predict(normal_test)

In [101]:
type(nn_pred)

numpy.ndarray

In [102]:
def convert():
    out = []
    output = []
    for i in range(len(nn_pred)):
        temp = max(nn_pred[i])
        for j in range(4):
            if temp == nn_pred[i][j]:
                out.append(j)
    for i in out:
        if i==0:
            output.append("Highly_Fatal_And_Damaging")
        if i==1:
            output.append("Minor_Damage_And_Injuries")
        if i==2:
            output.append("Significant_Damage_And_Fatalities")
        if i==3:
            output.append("Significant_Damage_And_Serious_Injuries")
    return output
                

In [103]:
nn_out = convert()

In [104]:
print(len(nn_out))

2500


In [105]:
submission = pd.DataFrame()
submission["Accident_ID"] = ID
submission["Severity"] = nn_out
submission.to_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth Airplane Accident/submission.csv",index = False)

In [ ]:
gbm.feature_importances_

In [ ]:
df.head()

In [ ]:
gb_imp = gbm.feature_importances_
imp_col = {}
for c,i in enumerate(df.columns):
    imp_col[gb_imp[c]] = i
print(imp_col)

In [ ]:
 x= sorted(imp_col.keys())

In [ ]:
for i in x:
    print(imp_col[i])

In [ ]:
for i in df.columns:
    print(df[i].nunique(),end= " ")
    print(i)

In [ ]:
df["Accident_Type_Code"].unique()

In [ ]:
(df['Accident_Type_Code'] == 1).sum()

In [ ]:
df.describe()

In [ ]:
df.values.tolist()[0]